# Load dependencies

In [1]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from sklearn import preprocessing #for one-hot encoding

For the Live SQL Server Authentication, please enter your


# SQL Script

In [12]:
s='''
    
        SELECT DISTINCT
        ST.[StudyID] as [StudyID]
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,[ICDChapter]

        FROM(
        SELECT 
        [StudyID]
        ,[ICDChapter]
        from [CPMS_BI].[dbo].[ODP_StudyICD]
        WHERE StudyID is NOT NULL) AS IC

        LEFT JOIN 
        (SELECT 
        StudyID
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,Study_IsCommercial
        ,Study_IsDraft

        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        ) AS ST
        ON IC.StudyID = ST.StudyID
        WHERE 1=1
        AND Study_IsCommercial = 0
        AND Study_IsDraft=0


        UNION

        SELECT  DISTINCT
        [StudyID]
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,'' as [ICDChapter]


        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        AND Study_IsCommercial = 0
        AND Study_IsDraft=0
        AND StudyID NOT IN (
        SELECT DISTINCT StudyID
        FROM [CPMS_BI].[dbo].[ODP_StudyICD]
        WHERE StudyID is NOT NULL
)
    '''

# with open("C:/Users/busaji/Documents/Projects/StudyClassifier-main/dat/sql-read.txt","r") as f:
#     s = f.read()

In [13]:
study_df = pd.read_sql(s, connection)

C:\Users\busaji\AppData\Local\Temp\ipykernel_21292\3074574563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [14]:
study_df.dtypes

StudyID                                        int64
StudyTitle                                    object
StudyShortName                                object
StudyResearchSummary                          object
StudyInclusionCriteria                        object
StudyDesignType                               object
StudyPhases_Concatenated                      object
StudySettings_Concatenated                    object
StudyManagingSpecialty                        object
StudyManagingSpecialty_PrimarySubSpecialty    object
ICDChapter                                    object
dtype: object

In [16]:
#pre-processing
study_df2 = study_df.fillna("not available")

#convert to float
#study_df['StudyResearchSummary'] = study_df['StudyResearchSummary'].astype("string")

In [17]:
study_df2

,StudyID,StudyTitle,StudyShortName,StudyResearchSummary,StudyInclusionCriteria,StudyDesignType,StudyPhases_Concatenated,StudySettings_Concatenated,StudyManagingSpecialty,StudyManagingSpecialty_PrimarySubSpecialty,ICDChapter
0,622,A Randomised Study of Radical Radiotherapy in ...,BS06,1. To determine the efficacy of radical radiot...,1. A new diagnosis of pT1G3 NXM0 tumour or tum...,Interventional,III,Tertiary Care,Cancer,Bladder Cancer,
1,623,A Randomised Trial of Paclitaxel (Taxol) in Co...,ICON4,To compare paclitaxel combined with platinum t...,1. Relapsed epithelial ovarian carcinoma previ...,Interventional,III,Primary Care,Cancer,Gynaecological Cancers,
2,624,A Multicentre Randomised Trial of Short Neo-Ad...,LY07,This is a randomized multicenter study. Patien...,1. Patients aged between 16 and 75 years 2. H...,Interventional,N/A,Primary Care,Cancer,Lymphoma,
3,625,A Randomised Trial to Evaluate Early High Dose...,LY02,A randomised trial to evaluate early high dose...,1. Age 16-65 years. 2. No medical conditions ...,Interventional,III,"Primary Care, Secondary Care",Cancer,Lymphoma,
4,626,"Randomised Clinical Trial of Ifosfamide, Carbo...",LU21,To compare an intensive regimen of VICE withou...,"1. Previously untreated, microscopically prove...",Interventional,II,Tertiary Care,Cancer,Lung Cancer,
...,...,...,...,...,...,...,...,...,...,...,...
27693,64977,Gestational Trophoblastic Disease/Neoplasia: e...,GTD-EXPLORE,"GTD is a serious complication of pregnancy, mo...",- Patients aged > = 16 years with a clinical/h...,Observational,N/A,Tertiary Care,Reproductive Health and Childbirth,Early Pregnancy,
27694,65005,PAIN P1; The Incidence of Neuroma and Neuropat...,PAIN P1 Post Amputation Incidence of Neuroma a...,Finger amputations are a common occurrence fol...,1. Patients who have had an upper limb digital...,Observational,N/A,Secondary Care,Surgery,Plastic and Hand Surgery,
27695,65057,Antiplatelet Secondary Prevention Internationa...,ASPIRING,Stroke due to intracerebral haemorrhage (ICH) ...,"• Stroke due to ICH, diagnosed by brain imagin...",Interventional,IV,Secondary Care,Dementias and Neurodegeneration,Other,
27696,65218,Exploring the perspectives of individuals usin...,Outreach services to identify cirrhosis: A qua...,Across the United Kingdom more people are dyin...,Individuals who have been recalled for their F...,Observational,N/A,Community,Public Health,Public Health,


In [20]:
#get study ID & ICD chapter
study_icd = study_df2[["StudyID","ICDChapter"]]

# One-Hot encoding for ICD Chapters

In [21]:
#load onehotencoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', 
                    categories='auto', 
                    drop='if_binary')

enc.fit(study_icd)


OneHotEncoder(drop='if_binary', handle_unknown='ignore')

In [ ]:
study_icd

In [24]:
#pandas one-hot encoding

df_encoded = pd.get_dummies(study_icd, dtype=int)

In [26]:
df_encoded.shape
#should be 23 shapes - 22 ICD Chapters and 1 blank

(27698, 23)

In [ ]:
df_encoded

In [30]:
#Merge source data and df_encoded together

data = pd.merge(
    left=study_df2,
    right=df_encoded,
    how='right',
    on='StudyID'
)

In [31]:
data.shape

(32838, 34)

# Pre-processing

In [33]:
#combine text data into one column
data['Text'] = data['StudyTitle'] + " " + data['StudyResearchSummary'] + " " + data['StudyInclusionCriteria']

In [34]:
#Setting StudyID as dataframe index
data.set_index('StudyID', inplace=True)

In [40]:
#drop unrequired fields

data_cln = data.drop(['StudyTitle','StudyResearchSummary','StudyInclusionCriteria',
            'StudyTitle', 'StudyShortName',
          'StudyDesignType','StudyPhases_Concatenated','StudySettings_Concatenated',
          'StudyManagingSpecialty','StudyManagingSpecialty_PrimarySubSpecialty'], axis=1)

In [41]:
data_cln = data.drop('StudyShortName', axis=1, inplace=True)

In [ ]:
X = data.drop('ICDChapter', axis=1)
y=data['ICDChapter']

#split data into categorical and numerical
#get list of categorical columns: (includes categorical where it is binary -1 & 0)
#ohe_col = data.select_dtypes(include=['object']).columns.tolist()
all_col = X.columns.tolist() #get list of all columns 
#num_col = all_col #instantiate a list of numerical columns 
num_col = X._get_numeric_data().columns
ohe_col = [col for col in all_col if col not in num_col] #getting list of one-hot enoding columns - categorical features
#loop to get list of remainder columns after excluding ohe columns:
# for a in all_col:
#   if a in ohe_col:
#     num_col.remove(a)
# X_ohe = X.drop(columns = ohe_col,inplace=False) #dataset for one-hot encoding transformation
# X_num = X.drop(columns = num_col,inplace=False) #dataset for with numerical columns 

#one-hot encoding
enc = preprocessing.OneHotEncoder(handle_unknown='ignore',sparse_output = False, drop = "if_binary" )

# 2. FIT
X_encoded = enc.fit(X[ohe_col])

# 3. Transform
#onehotlabels = enc.transform(X[ohe_col]).toarray()
onehotlabels = enc.transform(X[ohe_col])


#remove NAs or blanks
data = data.fillna(0, inplace=True)
#convert to float
data = data.astype(float)

# Convert to NumPy as required for k-fold splits
X_np = X.values
y_np = y.values

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_np, y_np, test_size = 0.25, random_state=42)

# Scale X data
X_train_sc, X_test_sc = pr.scale_data(X_train, X_test)


In [22]:
# onehotlabels = enc.transform(study_icd).toarray()
# onehotlabels.shape

(27698, 25605)

In [51]:
#workaround - but 'reload' is not defined
import sys
reload(sys)
sys.setdefaultencoding('utf8')

NameError: name 'reload' is not defined

1. Convert to string

In [60]:
#study_df2['StudyResearchSummary'] = study_df2['StudyResearchSummary'].encode('utf-8')

for index, row in study_df2.iterrows():
    row['StudyResearchSummary'] = row['StudyResearchSummary'].encode('utf-8')


# TF-IDF Vectorizer

In [61]:

corpus = study_df2['StudyResearchSummary']
vectorizer = TfidfVectorizer(lowercase = True, encoding = "utf-8", 
                             decode_error="ignore", stop_words='english', 
                             analyzer='word', ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X.shape)

(19513, 956142)


In [62]:
X.head()

AttributeError: 'csr_matrix' object has no attribute 'head'

# Multi-class Multi-Label Classfication

In [19]:
#load dependencies
from sklearn.datasets import make_multilabel_classification
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
def multi_label_classification(X_train, y_train, X_test):
    """
    This function builds and evaluates a multi-label classification model using Scikit-Learn.
    It trains a Logistic Regression model, and prints the classification report.
    """
    # Initialize the base classifier
    base_classifier = LogisticRegression(max_iter=1000)
    # Wrap the base classifier with MultiOutputClassifier
    multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)
    # Train the multi-label classifier
    multi_target_classifier.fit(X_train, y_train)
    # Predict the labels for the test set
    y_pred = multi_target_classifier.predict(X_test)
    # Evaluate the model
    report = classification_report(y_test, y_pred, target_names=[f'Label {i}' for i in range(y.shape[1])])
    # Print the classification report
    print(report)

In [ ]:
def multi_label_classification(X_train, y_train, X_test, y_test):
       # Convert text data to TF-IDF features
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    # Initialize the base classifier
    base_classifier = LogisticRegression(max_iter=1000)
    # Wrap the base classifier with MultiOutputClassifier
    multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)
    # Train the multi-label classifier
    multi_target_classifier.fit(X_train_tfidf, y_train)
    # Predict the labels for the test set
    y_pred = multi_target_classifier.predict(X_test_tfidf)
    # Evaluate the model
    report = classification_report(y_test, y_pred, target_names=[f'Label {i}' for i in range(len(y_train[0]))])
    # Print the classification report
    print(report)

# K-Means Clustering

In [63]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0, n_init="auto").fit(X)


In [64]:
feat_nms = kmeans.get_feature_names_out()

In [65]:
feat_nms

array(['kmeans0', 'kmeans1', 'kmeans2', 'kmeans3', 'kmeans4', 'kmeans5',
       'kmeans6', 'kmeans7', 'kmeans8', 'kmeans9', 'kmeans10', 'kmeans11',
       'kmeans12', 'kmeans13', 'kmeans14', 'kmeans15', 'kmeans16',
       'kmeans17', 'kmeans18', 'kmeans19'], dtype=object)

In [67]:
kmeans.

KMeans(n_clusters=20, random_state=0)

# LDA

In [66]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

lda = LatentDirichletAllocation(n_components=5,
    random_state=0)
lda.fit(X) #fit X


LatentDirichletAllocation(n_components=5, random_state=0)

In [68]:
#transform X
X_lda =lda.transform(X)



In [71]:
X_lda.shape

(19513, 5)

In [ ]:
#get output as pandas
X_lda.set_output(*,transform="pandas")